In [ ]:
import xarray as xr

In [2]:
# from dask_jobqueue import SLURMCluster
# cluster = SLURMCluster(scheduler_port=8789)
# cluster

In [3]:
# from distributed import LocalCluster
# cluster = LocalCluster(n_workers=3,threads_per_worker=1)
# cluster

In [4]:
from distributed import Client
# client=Client(cluster)
client=Client('tcp://localhost:8789')
client

Client Scheduler: tcp://localhost:8789 Dashboard: http://localhost:8787/status,Cluster Workers: 15 Cores: 30 Memory: 360.00 GB


### 4. Against the Zarr on local (Lustre - fast!) file system 

Notice how each time the number of lines of code is reducing, better chunking in the zarr probably helps in this. Most of the time is now in building the dask task graph and copying the dataset back to the memory of the notebook! The calculation only takes < 5 s (!)

In [5]:
import os
zarr_path = os.environ['MYSCRATCH'] + '/aqua.P1D.aust.K_490.Full.zarr'
ds_zarr = xr.open_zarr(zarr_path) # Already chunked on filesystem
ds_zarr

<xarray.Dataset>
Dimensions:    (latitude: 7001, longitude: 10001, time: 4882)
Coordinates:
  * latitude   (latitude) float64 10.0 9.99 9.98 9.97 ... -59.98 -59.99 -60.0
  * longitude  (longitude) float64 80.0 80.01 80.02 80.03 ... 180.0 180.0 180.0
  * time       (time) datetime64[ns] 2002-07-06 2002-07-07 ... 2016-12-31
Data variables:
    K_490      (time, latitude, longitude) float32 dask.array<shape=(4882, 7001, 10001), chunksize=(7, 856, 1223)>
Attributes:
    Conventions:  CF-1.6
    history:      File initialised at 2015-12-16T15:54:44.125153\nInitialised...
    source_path:  imos-srs/archive/oc/aqua/1d/v201508/2002/07/A20020706.L2OC_...

In [6]:
import numpy as np
numel = np.prod(ds_zarr.K_490.shape)
print('14 years of daily analysis ready satellite data from IMOS-SRS team with complete australian coverage')
print('Dataset size %.1f billion array elements, %.2f GB' % (numel/1E9,ds_zarr.nbytes/1E9))

14 years of daily analysis ready satellite data from IMOS-SRS team with complete australian coverage
Dataset size 341.8 billion array elements, 1367.29 GB


### Calculate anomalies from a climatology for a region of interest

In [7]:
cropped = ds_zarr.sel(longitude=slice(113,117),latitude=slice(-20,-23))

month_group = cropped.groupby('time.month')
month_mean = month_group.mean('time')
month_std = month_group.std('time')

# persist these calculations to the cluster
# kicks off the calculation and then retains the result spread across the workers for later use
month_mean, month_std = client.persist([month_mean, month_std])

Assume that the light attenuation is gaussian distributed, define an anomoly as 2.5 times the standard deviation above the mean (possibly a poor assumption!)

In [8]:
month_group_2014 = cropped.sel(time=slice('2013-01-01', '2014-01-01')).groupby('time.month')
month_mean_2014 = month_group_2014.mean('time')
excess_2014 = month_mean_2014 - month_mean
threshold = month_mean_2014 - (month_mean + 2.5*month_std)

excess_2014, threshold = client.persist([excess_2014, threshold])

### Plot the results in the browser with holoviews+geoviews

In [ ]:
import holoviews as hv
import hvplot.xarray
import hvplot.pandas
import geoviews as gv
import numpy as np
import cartopy
from cartopy import crs as ccrs

hv.notebook_extension('bokeh')

In [10]:
plate=ccrs.PlateCarree(central_longitude=133.5)

In [ ]:
%%opts Overlay [width=700 height=400]
%%opts Image [projection=plate]

thres_dataset = gv.Dataset(threshold, kdims=['month', 'longitude', 'latitude'], vdims=['K_490'])

(excess_2014.K_490.hvplot('longitude','latitude',groupby='month',dynamic=True, rasterize=True, cmap='Oranges',crs=ccrs.PlateCarree()).redim(K_490=dict(range=(0, 0.2))))*\
(gv.feature.coastline.geoms('10m')).opts(color='black')
